# Project 2


In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  #To get figures with high quality!

import numpy as np
import torch
from torch import nn
from torch import optim
import matplotlib.pyplot as plt
import time
import torchvision

## Loading the dataset CIFAR10

We are told to classify birds from cats in the CIFAR10 dataset. This means that we will only use the images that are labeled as birds or cats. In the CIFAR10 these labels are 2 and 3 respectively. We will have the training, validation and test datasets.

In [74]:
from torchvision import datasets, transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

#Filtering the birds and cats images from the train set
trainset.targets = torch.tensor(trainset.targets)
mask = (trainset.targets == 2) | (trainset.targets == 3)
trainset.targets = trainset.targets[mask]
trainset.data = trainset.data[mask.numpy().astype(bool)]

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)


testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

#Filtering the birds and cats images from the test set
testset.targets = torch.tensor(testset.targets)
mask = (testset.targets == 2) | (testset.targets == 3)
testset.targets = testset.targets[mask]
testset.data = testset.data[mask.numpy().astype(bool)]

testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


## Implementation of a Lenet5

In [75]:
class Lenet5(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_part = nn.Sequential( # As images are rgb we have 3 input channels
            nn.Conv2d(in_channels=3, out_channels=6,
                      kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120), # The input dimension at the classifier is 16 images of 5x5 so 16*5*5
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
            nn.Sigmoid() # Output layer with a probability for being a bird or a cat
        )
        self.lr = 0.01 #Learning Rate

        self.optim = optim.Adam(self.parameters(), self.lr)

        self.criterion = nn.NLLLoss()

        self.loss_during_training = []

        self.valid_loss_during_training = []


    def forward(self, x):
        x = self.conv_part(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def trainloop(self,trainloader,validloader,epochs):

        self.train()
        for e in range(epochs):

            running_loss = 0.

            for images, labels in trainloader:

                self.optim.zero_grad()

                out = self.forward(images)


                loss = self.criterion(out,labels)

                running_loss += loss.item()


                loss.backward()
                self.optim.step()


            self.loss_during_training.append(running_loss/len(trainloader))

            # Validtion loss

            # Deactivate the gradients
            with torch.no_grad():

                self.eval()

                running_loss = 0.

                for images,labels in validloader:

                    # Batch output
                    out = self.forward(images)

                    #Loss function
                    loss = self.criterion(out,labels)

                    running_loss += loss.item()

                self.valid_loss_during_training.append(running_loss/len(validloader))

                self.train()

                print("Epoch %d. Training loss: %f, Validation loss: %f"
                      %(e,self.loss_during_training[-1],self.valid_loss_during_training[-1]))

In [77]:
model = Lenet5()
model.trainloop(trainloader,testloader,10)

Epoch 0. Training loss: -0.994691, Validation loss: -1.000000
Epoch 1. Training loss: -1.000000, Validation loss: -1.000000
Epoch 2. Training loss: -1.000000, Validation loss: -1.000000
Epoch 3. Training loss: -1.000000, Validation loss: -1.000000
Epoch 4. Training loss: -1.000000, Validation loss: -1.000000
Epoch 5. Training loss: -1.000000, Validation loss: -1.000000


KeyboardInterrupt: 